### H-Pile Design 
#### Per NHI Courses No. 132021 and 132022 "Design and Construction of Driven Pile Foundations - Volume 1 & 2"
####     "LRFD Bridge Design Specifications" 9th Edition 2020 

In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Button
from IPython.display import display
from ipyfilechooser import FileChooser
import io
import json

#### Pile Structural Information

In [ ]:
#load csv with all the H-Pile info
H_Pile = pd.read_csv("https://raw.githubusercontent.com/geotechnick/Geotechnical_Design/main/Pile_Design/H_Piles/HP_Spec_Table.csv")
Fy = 50 # Pile yield stress in ksi
pile_names = H_Pile['Section_E'].unique().tolist()

#filter out the metric values
H_Pile = H_Pile.loc[:, ~H_Pile.columns.str.endswith('_M')]

#create drop down with the pile names 
dropdown = widgets.Dropdown(options=pile_names, description='Select Value:')

# Function to display the selected row 
def display_row(selected_value):
    global design_pile
    design_pile = H_Pile[H_Pile['Section_E'] == selected_value] #creates dataframe that only has the selected value in it
    pile_units = H_Pile[H_Pile['Section_E'] == 'Units_English'] #create separate dataframe that has units
    pile_display = pd.concat([design_pile, pile_units], ignore_index=True) #combine pile properties and pile units into single dataframe
    display(pile_display.T)

# Connect the dropdown widget to the display function using interact
widgets.interact(display_row, selected_value=dropdown) 

#### Upload Soil Profile

In [ ]:
# Function to handle file upload
def handle_upload(change):
    uploaded_file = change['new'][0]
    content = uploaded_file['content']
    content_bytes = io.BytesIO(content)
    df = pd.read_csv(content_bytes)

    # For now, let's just print the first few rows
    print(df.head())
    # Save the dataframe as a global variable so it can be used outside of the function 
    global layer_params
    layer_params = df

# Creating file upload widget
file_upload = widgets.FileUpload(accept='.csv')

# Attaching the event handler
file_upload.observe(handle_upload, names='value')

# Displaying the widget
display(file_upload)

In [ ]:
layer_params

### Save properties to JSON

In [ ]:
def save_json_to_file(json_data, file_path):
    if not file_path.endswith('.json'):
        file_path += '.json'
    with open(file_path, 'w') as json_file:
        json_file.write(json_data)
    print(f"JSON data saved to {file_path}")

def save_json_widget(json_data):
    file_chooser = FileChooser()
    file_chooser.use_dir_icons = True
    file_chooser.title = "Select a location to save the JSON file"
    save_button = widgets.Button(description="Save JSON")

    def save_json(button):
        if file_chooser.selected:
            save_json_to_file(json_data, file_chooser.selected)
        else:
            print("Please select a valid file path.")

    save_button.on_click(save_json)
    return widgets.VBox([file_chooser, save_button])

# Create JSON File ###############################
# Convert DataFrames to dictionaries
dict1 = design_pile.to_dict(orient='records')
dict2 = layer_params.to_dict(orient='records')

# Create a dictionary with dictionaries
pile_dict = {'Pile_Info': dict1, 'Soil_Properties': dict2}

# Specify the file path where you want to save the JSON file
file_path = "C:/Users/millern8989/OneDrive - ARCADIS/Documents/Python Calculations/dataframes.json"

# Convert the dictionary to JSON format
json_data = json.dumps(pile_dict, indent=4)
###################################################

# Display the widget
save_json_widget(json_data)

### Upload from save file


In [ ]:
def handle_upload(change):
    uploaded_file = change['new'][0]
    content = uploaded_file['content']
    content_bytes = content.tobytes()
    content_str = content_bytes.decode('utf-8')
    data = json.loads(content_str)

    # Check if 'Pile_Info' key exists and create a DataFrame
    if 'Pile_Info' in data:
        pile_info_df = pd.DataFrame(data['Pile_Info'])
        print("DataFrame for 'Pile_Info':")
        print(pile_info_df)
        # Optionally, you can save the DataFrame to a file
        # pile_info_df.to_csv("pile_info_data.csv", index=False)
    else:
        print("The key 'Pile_Info' does not exist in the JSON data.")

    # Check if 'Soil_Properties' key exists and create a DataFrame
    if 'Soil_Properties' in data:
        soil_properties_df = pd.DataFrame(data['Soil_Properties'])
        print("\nDataFrame for 'Soil_Properties':")
        print(soil_properties_df)
        # Optionally, you can save the DataFrame to a file
        # soil_properties_df.to_csv("soil_properties_data.csv", index=False)
    else:
        print("The key 'Soil_Properties' does not exist in the JSON data.")

# Creating file upload widget
file_upload = widgets.FileUpload(accept='.json')

# Attaching the event handler
file_upload.observe(handle_upload, names='value')

# Displaying the widget
display(file_upload)

In [ ]:
class pile_design:
    
    def __init__(self):
        """Initialize the PileDesign object.

    Args:
        pile_specs (pandas.DataFrame): A DataFrame containing specifications for the pile design.
            This DataFrame should have columns representing H-pile properties such as web dimensions, thickness, and interia.
        soil_params (pandas.DataFrame): A DataFrame containing parameters describing the soil layers.
            This DataFrame should have columns representing soil properties such as depth, unit weight, cohesion, and friction angle for each layer.
    """
        self.pile_specs = design_pile
        self.soil_params = layer_params
        